<a href="https://colab.research.google.com/github/LeaslyG/proyecto_ap_refuerzo/blob/master/DQN_LAP_100000steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/drive'
drive_root = mount + "/My Drive/content/proyecto3"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

In [5]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

We're running Colab
Colab: mounting Google drive on  /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Colab: making sure  /content/drive/My Drive/content/proyecto3  exists.

Colab: Changing directory to  /content/drive/My Drive/content/proyecto3
/content/drive/My Drive/content/proyecto3
Archivos en el directorio: 
[]


In [3]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.5 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654618 sha256=ca0c7da146e263e24e1b5473aa7534ab3e4dda4719977511fdbd5c69830dafc8
  Stored in directory: /root/.cache/pip/wheels/af/4b/74/fcfc8238472c34d7f96508a63c962ff3ac9485a9a4137afd4e
Successfully built gym
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.2.1
    Uninstalling cloudpickle-2.2.1:
      Successfully uninstalled cloudpickle-2.2.1
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.8.0 requires cloudpick

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!pip install gym
import gym

In [8]:
# Creamos el entorno sobre el que vamos a ejecutar nuestro agente
env = gym.make("BreakoutDeterministic-v4")
env.seed(123)  # Definimos la semilla para la reproducibilidad de la ejecución

[123, 151010689]

In [11]:
if IN_COLAB:
  !pip install gym[atari-py]
  !apt-get install -y xvfb ffmpeg
  !pip install imageio

  import gym
  import imageio
  import numpy as np
  import matplotlib.pyplot as plt
  from IPython.display import HTML
  from base64 import b64encode

  # Función para crear un video a partir de imágenes
  def create_video(frames, video_name):
      imageio.mimsave(video_name, frames, fps=30)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 45 not upgraded.
Need to get 7,813 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main a

In [14]:
from __future__ import division

from PIL import Image
import numpy as np
import gym
from gym.wrappers import AtariPreprocessing, FrameStack

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Input, concatenate, Lambda, Conv2D
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
from rl.core import Processor

In [15]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

In [16]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channels)
        if observation.shape[2] == 3:
            observation = np.mean(observation, axis=2).astype(np.uint8)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        # Replicar la observación procesada a través de los 4 canales
        processed_observation = np.stack([processed_observation] * WINDOW_LENGTH, axis=2)
        assert processed_observation.shape == (INPUT_SHAPE[0], INPUT_SHAPE[1], WINDOW_LENGTH)
        return processed_observation

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        # Eliminar la dimensión adicional para tener la forma correcta
        if processed_batch.ndim == 5:
            processed_batch = np.squeeze(processed_batch, axis=1)
        return processed_batch

    def process_reward(self, reward):
        print("Original reward:", reward)
        processed_reward = np.clip(reward, -1., 1.)
        print("Processed reward:", processed_reward)
        return processed_reward

In [17]:
class ObservationReshaper(gym.ObservationWrapper):
    def __init__(self, env):
        super(ObservationReshaper, self).__init__(env)
        self.observation_space = gym.spaces.Box(
            low=0, high=255, shape=(INPUT_SHAPE[0], INPUT_SHAPE[1], WINDOW_LENGTH), dtype=np.uint8)

    def observation(self, obs):
        # Reorganiza la observación de (WINDOW_LENGTH, 84, 84, 1) a (84, 84, WINDOW_LENGTH)
        if obs.shape == (WINDOW_LENGTH, INPUT_SHAPE[0], INPUT_SHAPE[1], 1):
            obs = np.squeeze(obs, axis=-1)  # Elimina la dimensión redundante
            obs = np.transpose(obs, (1, 2, 0))  # Transpone para obtener la forma correcta
        return obs

In [ ]:
def construct_q_network(num_actions,dim=(84,84,4), loss='mse', optimizer=Adam(lr=0.00001)):
    # Uses the network architecture found in DeepMind paper
    model = Sequential()
    model.add(Convolution2D(32, (8, 8), strides=(4, 4), input_shape=dim))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(num_actions))
    model.compile(loss=loss, optimizer=optimizer)
    print("Successfully constructed networks.")
    return model

In [ ]:
env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)
env = AtariPreprocessing(env, frame_skip=1, grayscale_newaxis=True)
env = FrameStack(env, num_stack=WINDOW_LENGTH)
env = ObservationReshaper(env)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n
print("Numero de acciones disponibles: " + str(nb_actions))
print("Formato de las observaciones: " + str(env.observation_space))

In [ ]:
memory = SequentialMemory(limit=50000, window_length=1)

In [ ]:
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps',value_max=1, value_min=0.1, value_test=0.2, nb_steps=50000)

In [ ]:
model= construct_q_network(nb_actions, dim=(INPUT_SHAPE[0], INPUT_SHAPE[1], WINDOW_LENGTH), loss='mse', optimizer=Adam(lr=1e-3))
model.summary()
processor = AtariProcessor()

dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory,processor=processor,
               nb_steps_warmup=1000,gamma=0.9,
               target_model_update=1e-2, policy=policy) #target model te congela durante 100 steps lo que se usa para el "ground truth"
               #el warmup hace una policy aleatoria para ir descubriendo estados y viendo como furrula aprovechando que hay exp replay

dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])


In [ ]:
import os
print("Archivos en el directorio actual:")
print(os.listdir('.'))
# Cargar los pesos desde el checkpoint
latest_checkpoint = 'dqn_SpaceInvaders-v0_weights_10000.h5f'  # Cambia esto al nombre del último checkpoint guardado
if latest_checkpoint in os.listdir('.'):
    dqn.load_weights(latest_checkpoint)
    print("Checkpoint loaded successfully.")
else:
    print(f"Checkpoint {latest_checkpoint} not found.")

In [58]:
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
rewards_filename = 'dqn_{}_rewards.json'.format(env_name)
# Define los callbacks
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=25000)]
callbacks += [FileLogger(log_filename, interval=10)]

sample_observation = env.reset()
print(f"Sample observation shape: {np.array(sample_observation).shape}")


Sample observation shape: (84, 84, 4)


In [60]:
print("Starting training...")
dqn.fit(env, callbacks=callbacks, nb_steps=100000, log_interval=10000, visualize=False)
dqn.save_weights(weights_filename, overwrite=True)

Se han truncado las últimas 5000 líneas del flujo de salida.
 7501/10000 [=====================>........] - ETA: 10:32 - reward: 0.0185Original reward: 0.0
Processed reward: 0.0
 7502/10000 [=====================>........] - ETA: 10:32 - reward: 0.0185Original reward: 0.0
Processed reward: 0.0
 7503/10000 [=====================>........] - ETA: 10:32 - reward: 0.0185Original reward: 0.0
Processed reward: 0.0
 7504/10000 [=====================>........] - ETA: 10:31 - reward: 0.0185Original reward: 0.0
Processed reward: 0.0
 7505/10000 [=====================>........] - ETA: 10:31 - reward: 0.0185Original reward: 0.0
Processed reward: 0.0
 7506/10000 [=====================>........] - ETA: 10:31 - reward: 0.0185Original reward: 0.0
Processed reward: 0.0
 7507/10000 [=====================>........] - ETA: 10:31 - reward: 0.0185Original reward: 0.0
Processed reward: 0.0
 7508/10000 [=====================>........] - ETA: 10:30 - reward: 0.0185Original reward: 0.0
Processed reward: 0.0
 75

In [62]:

weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Se han truncado las últimas 5000 líneas del flujo de salida.
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed reward: 0.0
Original reward: 0.0
Processed rewar

In [66]:
import json
import numpy as np
log_filename = 'dqn_{}_log.json'.format(env_name)

with open(log_filename, 'r') as f:
    log_data = json.load(f)
episode_rewards = log_data.get('episode_reward', [])
print("Processed rewards:", episode_rewards)

if episode_rewards:
    max_processed_reward = np.max(episode_rewards)
    print("Max processed reward:", max_processed_reward)
else:
    print("No rewards to display.")

Processed rewards: [7.0, 17.0, 8.0, 6.0, 6.0, 11.0, 5.0, 11.0, 11.0, 6.0, 10.0, 18.0, 15.0, 3.0, 12.0, 6.0, 13.0, 7.0, 6.0, 21.0, 8.0, 8.0, 5.0, 25.0, 19.0, 11.0, 7.0, 7.0, 15.0, 19.0, 5.0, 14.0, 10.0, 9.0, 13.0, 5.0, 5.0, 9.0, 10.0, 19.0, 15.0, 14.0, 12.0, 8.0, 22.0, 12.0, 5.0, 26.0, 19.0, 16.0, 17.0, 19.0, 22.0, 19.0, 16.0, 24.0, 5.0, 6.0, 20.0, 9.0, 4.0, 20.0, 21.0, 15.0, 24.0, 21.0, 6.0, 14.0, 24.0, 20.0, 15.0, 12.0, 10.0, 16.0, 9.0, 11.0, 11.0, 11.0, 6.0, 19.0, 6.0, 8.0, 23.0, 13.0, 24.0, 7.0, 12.0, 26.0, 9.0, 11.0, 7.0, 14.0, 10.0, 9.0, 13.0, 16.0, 7.0, 11.0, 18.0, 16.0, 20.0, 13.0, 10.0, 15.0, 27.0, 11.0, 26.0, 6.0, 18.0, 6.0, 28.0, 27.0, 15.0, 14.0, 14.0, 11.0, 10.0, 10.0, 19.0, 12.0, 16.0, 29.0, 12.0, 10.0, 6.0]
Max processed reward: 29.0
